# Docking molecules into a pre-defined binding site

In previous tutorials, you should have learned how to generate the full 3D structures starting from a given SMILES string. Now, we will take those 3D molecules and dock them into a pre-prepared receptor file that has the protein binding site already defined. To prepare a receptor file for docking, see the [MAKE_RECEPTOR](https://docs.eyesopen.com/oedocking/make_receptor_gui.html) docs for the OpenEye Toolkits. This particular example uses a prepared receptor of soluble epoxide hydrolase that we have been working on for research purposes.

In this tutorial we will be using the docking method [FRED](https://docs.eyesopen.com/oedocking/fred.html) which is used in cases where the protein does not have a bound ligand, but we define a region or box to dock our molecules to.

In [1]:
#Import our required openeye modules
from openeye import oechem, oedocking

In [2]:
#Define file path and initialize receptor variable
receptor_file = 'sEH-receptor.oeb'
receptor = oechem.OEGraphMol()

#Read in our receptor from disc
if not oedocking.OEReadReceptorFile( receptor, str( receptor_file ) ):
    # raise an exception if the receptor file cannot be read
    raise Exception("Unable to read receptor from {0}".format( receptor_file ))

In [3]:
#Set the docking method and other paramters
# Note: Chemgauss4 is the scoring function for FRED
dock_method = oedocking.OEDockMethod_Chemgauss4
dock_resolution = oedocking.OESearchResolution_Default
sdtag = oedocking.OEDockMethodGetName( dock_method )

#Generate our OEDocking object
dock = oedocking.OEDock( dock_method, dock_resolution)

#Initialize the OEDocking by providing it the receptor
if not dock.Initialize(receptor):
    # raise an exception if the receptor cannot be initialized
    raise Exception("Unable to initialize Docking with {0}".format(self.args.receptor))

Now that we have initialized our OEDocking object with our receptor, let's write a function that will take in the following input parameters:
    - dock: OEDock object
    - sdtag: string representing the name of the docking method
    - numpose: int with the number of poses to generate for each ligand
    - mcmol: multicomformer molecule

In [4]:
def dock_molecule( dock: "OEDock", sdtag: str, num_poses: int, mcmol ) -> tuple:
    ''' Docks the multiconfomer molecule, with the given number of poses
        Returns a tuple of the docked molecule (dockedMol) and its score
        i.e. ( dockedMol, score )
    '''
    dockedMol = oechem.OEMol()

    #Dock the molecule into a given number of poses
    res = dock.DockMultiConformerMolecule(dockedMol, mcmol, num_poses)
    
    if res == oedocking.OEDockingReturnCode_Success:
        
        #Annotate the molecule with the score and SDTag that contains the docking method
        oedocking.OESetSDScore(dockedMol, dock, sdtag)
        dock.AnnotatePose(dockedMol)
        score = dock.ScoreLigand(dockedMol)
        oechem.OESetSDData(dockedMol, sdtag, "{}".format(score))
        return dockedMol, score
    
    else:
        # raise an exception if the docking is not successful
        raise Exception("Unable to dock ligand {0} to receptor".format( dockedMol ))

With the docking function written, we can then loop over our 3D molecules and dock them to the given receptor

In [ ]:
#Define how many docked poses to generate per molecule
num_poses = 2

#Read in our 3D molecules 
with oechem.oemolistream('c0-chgd.oeb.gz') as ifs:
    
    #Open a filestream for writing the docked molecules
    with oechem.oemolostream( 'c0-dockd.oeb.gz') as ofs:
        
        #Loop over 3D molecules from the input filestream
        for mcmol in ifs.GetOEMols():
            
            #Call our written docking function
            dockedMol, score = dock_molecule( dock, sdtag, num_poses, mcmol )
            print("{} {} score = {:.4f}".format(sdtag, dockedMol.GetTitle(), score))
            
            #Write docked molecules to output filestream
            oechem.OEWriteMolecule(ofs, dockedMol)

Chemgauss4 V2Z_1 score = -11.0289
Chemgauss4 6NM_2 score = -13.2391
Chemgauss4 6NM_1 score = -13.2391
Chemgauss4 KWB_1 score = -10.9041
Chemgauss4 6NJ_1 score = -11.7682
Chemgauss4 6NJ_2 score = -12.3281
Chemgauss4 FCW_1 score = -11.5328
Chemgauss4 BSU_1 score = -11.2829
Chemgauss4 WMR_1 score = -10.0978
Chemgauss4 5ZM_1 score = -11.9286
Chemgauss4 6NF_1 score = -10.8372
Chemgauss4 WMR_2 score = -11.9275
Chemgauss4 GZP_1 score = -12.4088
Chemgauss4 6NF_2 score = -10.8372


# Visualizing the ligands

To visualize the ligands in their docked poses, you can use OpenEye's visualization application called [VIDA](https://www.eyesopen.com/vida) if you download and install it; alternatively you can write out your receptor to a PDB file and your ligands to files (such as mol2 files) and visualize with another viewer such as PyMol or Chimera. If you wish to use Vida, click "support -> downloads" at the top, fill in your information, and click "downloads"; find the appropriate version of Vida and install it. Using it will require your OpenEye license to be installed.

Assuming you are using Vida, if you want to view ligands within the binding site you will need to load both the docked molecules file and the receptor file.
> vida c0-dockd.oeb.gz  sEH-receptor.oeb


## Changing the view settings
If the protein is not being displayed at first, simply toggle the green button from the list view menu (shown below):
![Toggle](imgs/listview.png)


I like to turn on the ribbon representation for the protein (shown below):
![Ribbons](imgs/ribbons.png)


Now, if you toggle the green button from the list view for the ligands, you will see the 2 docked poses generated for the selected ligand. You can also hit `Center` to move the ligand into the center view.
![Ligands](imgs/listligand.png)
![6NJ_1-poses](imgs/6NJ_1-poses.png)


# Acknowledgements

- Authors: Meghan Osato, Nathan M. Lim
- Edited by D. Mobley